In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pydeck as pdk

# 포스트 데이터 불러오기
df_post = pd.read_csv('../inputs/posts/itewon.csv', index_col=0)

# 텍스트 없으면 drop
df_post = df_post.dropna()

# 12월로 필터링
df_post = df_post.loc[df_post.loc[:,'date'].str.contains('2023-12')]

In [4]:
# 상점 데이터 불러오기
gdf_store_itewon = gpd.read_file('../inputs/stores/store_itewon.geojson')

# 음식점만 필터링
gdf_store_itewon = gdf_store_itewon.loc[gdf_store_itewon.loc[:,'상권업종대분류명'] == '음식']

# 상호명 없는 상점은 삭제
gdf_store_itewon = gdf_store_itewon.dropna(subset='상호명')

# 상호명과 geometry만 남기고 다 삭제 (현재 필요없는 컬럼들)
gdf_store_itewon = gdf_store_itewon.loc[:,['상호명', 'geometry']]

# 상호명: store_name으로 변경
gdf_store_itewon = gdf_store_itewon.rename(columns={'상호명':'store_name'})

In [5]:
gdf_store_itewon.shape

(640, 2)

In [6]:
def counting_post(store_name):
    '''
    상점 이름을 input으로 받아 posting 갯수를 세주는 함수

    input:
        store_name (string): 상호명
    output:
        posting_count (int): 포스팅 갯수

    '''

    # 상호명이 한글자이면 그냥 0개로 취급
    if len(store_name) == 1:
        return 0

    # 포스트 중에서 상점 이름이 포함되어 있으면 좌표 추가
    posting_count = df_post.loc[df_post.loc[:,'text'].str.contains(store_name, regex=False)].shape[0]

    return posting_count


In [7]:
gdf_store_itewon.loc[:,'posting_count'] = gdf_store_itewon.loc[:,'store_name'].apply(counting_post)

In [8]:
# 포스팅 갯수 기준으로 정렬
gdf_store_itewon = gdf_store_itewon.sort_values(by='posting_count', ascending=False)

In [9]:
for i, row in gdf_store_itewon.iterrows():
    print(row['store_name'], row['posting_count'])
    if row['posting_count'] == 0:
        break

태원 509
갤러리 118
하지 40
로우 25
이드 20
이클래스 18
버터 15
문화 15
플레이 12
로드 9
일로 7
러브하우스 5
쿠키 5
부다스벨리 4
부다스벨리 4
옥탑 3
소울트레인 3
코끼리 3
그루브 3
아지트 2
신세계 2
농담 2
로즈앤크라운 2
코이 2
릴리 2
애플 2
왕타이 2
포즈 2
플랜트 2
레트로 2
보리 2
피치 2
님부스 2
누에 2
온도 2
루아 2
소서원 1
모터시티바이매니멀 1
하바나 1
매니멀스모크하우스 1
골드 1
바부 1
샘라이언 1
이태원소주집 1
이즈비 1
펀치 1
크리스피포크타운 1
야상해 1
술꼬마 1
아틀리에 1
시바 1
브라이리퍼블릭푸드 1
이태원맥주집 1
더뷰 1
이태원양꼬치 1
몽키 1
프리즘 1
이태원교집합 1
게더링 1
코지 1
민들레 1
핌피 1
챔프커피 1
헵시바극장 1
매덕스피자 1
나이트사운즈 1
호지보보 1
리지트커피 1
노머시버거 1
헤미안커피바 1
모에드 1
유니콘 1
프리츠아르투아 1
로우앤슬로우 1
핌트 1
코레아노스 1
데모 1
스태프온리 1
모여라 1
스카이 1
리저브 1
베티 1
까를로스 1
노티 1
멘야산다이메 1
다모토리 1
테이프 1
욱이족발 1
한국맥도날드이태원점 0


In [10]:
list_stopwords = [
'태원',
'크리스',
'갤러리',
'예인',
'맛집',
'리단',
'원타',
'하지',
'레스',
'하우스',
'포토',
'커피',
'스타일',
'이태원포토'
]

In [11]:
# list_stopwords 안에 있는 상점들은 우선 데이터프레임에서 제외
gdf_store_itewon = gdf_store_itewon.loc[~gdf_store_itewon.loc[:,'store_name'].isin(list_stopwords)]

# 포스팅 수가 0인 것들도 데이터프레임에서 제외
gdf_store_itewon = gdf_store_itewon.loc[gdf_store_itewon.loc[:,'posting_count']>0]

In [12]:
gdf_store_itewon

,store_name,geometry,posting_count
1429,로우,POINT (126.98959 37.53339),25
1431,이드,POINT (126.99640 37.53325),20
1181,이클래스,POINT (126.99583 37.53410),18
671,버터,POINT (126.99204 37.53505),15
908,문화,POINT (126.99388 37.53389),15
...,...,...,...
1502,노티,POINT (126.99682 37.53438),1
822,멘야산다이메,POINT (126.99704 37.53453),1
1142,다모토리,POINT (126.99521 37.53390),1
1248,테이프,POINT (126.99086 37.53344),1


In [13]:
gdf_store_itewon.loc[:,'posting_count'].sum()

237

In [14]:
# pydeck layer 설정

scatterplot_layer = pdk.Layer(
    'ScatterplotLayer',
    gdf_store_itewon,
    pickable=True,
    opacity=0.3,
    stroked=False,
    filled=True,
    radius_scale=2,
    get_position='geometry.coordinates',
    get_radius='posting_count',
    get_fill_color=[255,140,0]
)

In [15]:
# Set the viewport location
view_state = pdk.ViewState(latitude=37.5385, longitude=126.9925, zoom=15, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[scatterplot_layer], initial_view_state=view_state, tooltip={"text": "{store_name}\n{posting_count}"})
r.to_html("../output/maps/itewon/scatterplot_layer.html")

In [16]:
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=gdf_store_itewon,
    opacity=0.5,
    get_position='geometry.coordinates',
    aggregation=pdk.types.String("MEAN"),
    get_weight="posting_count",
)

In [17]:
r = pdk.Deck(
    layers=[heatmap_layer],
    initial_view_state=view_state
)

r.to_html("../output/maps/itewon/heatmap_layer.html")